# Data Ingestion Stage

In [27]:
from collections import namedtuple
import os

In [40]:
os.chdir("/Users/kartiknerwal/Git-Repos/cnnClassifier")

In [42]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [43]:
# from dataclasses import dataclass
# from pathlib import Path

# @dataclass(frozen=True)
# class DataIngestionConfig:
#     root_dir: Path
#     source_URL: str
#     local_data_file: Path
#     unzip_dir: Path

In [44]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [45]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [46]:
import os
import urllib.request as request
from zipfile import ZipFile

In [47]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)



    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)

            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)




In [48]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2023-07-29 20:51:08,636: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-07-29 20:51:08,637: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 20:51:08,638: INFO: common: created directory at: artifacts]
[2023-07-29 20:51:08,639: INFO: common: created directory at: artifacts/data_ingestion]


HTTPError: HTTP Error 404: Not Found